In [2]:
import numpy as np
import pandas as  pd
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [3]:
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')

In [4]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [5]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.columns

Index(['v1', 'v2'], dtype='object')

In [7]:
x=df['v2']
y=df['v1']

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
le=LabelEncoder()

In [10]:
y=le.fit_transform(y)

In [11]:
max_len=150
max_word=1000

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=232)

In [14]:
type(x_train)

pandas.core.series.Series

In [15]:
token=Tokenizer(num_words=1000,filters='"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

In [16]:
token.fit_on_texts(x_train)

In [17]:
sequences=token.texts_to_sequences(x_train)
sequencetest=token.texts_to_sequences(x_test)

In [18]:
matrix=sequence.pad_sequences(sequences,maxlen=max_len)
matrix_test=sequence.pad_sequences(sequencetest,maxlen=max_len)

In [19]:
import tensorflow.keras as k


In [20]:
def lstm_model():
    inp=k.layers.Input(shape=[max_len],name='input_layer')
    i=k.layers.Embedding(1000,50,input_length=max_len,mask_zero=True)(inp)
    
    i=k.layers.LSTM(64,name='lstm_1',return_sequences=True)(i)
    i=k.layers.LSTM(64,name='lstm_2',return_sequences=False)(i)
    i=k.layers.Dense(512,name='fc1')(i)
    i=k.layers.Activation('relu')(i)
    i=k.layers.Dropout(0.5)(i)
    i=k.layers.Dense(1,activation='sigmoid')(i)
    model=k.models.Model(inputs=inp,outputs=i)
    return model
    
    

In [21]:
model=lstm_model()

In [22]:
model = lstm_model()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 64)           29440     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
fc1 (Dense)                  (None, 512)               33280     
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0   

In [23]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 64)           29440     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
fc1 (Dense)                  (None, 512)               33280     
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0   

In [24]:
model.fit(matrix,y_train,batch_size=128,epochs=5,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 4011 samples, validate on 1003 samples
Epoch 1/5
4011/4011 [==============================] - 31s 8ms/sample - loss: 0.2790 - accuracy: 0.9122 - val_loss: 0.0908 - val_accuracy: 0.9711
Epoch 2/5
4011/4011 [==============================] - 15s 4ms/sample - loss: 0.0571 - accuracy: 0.9865 - val_loss: 0.0660 - val_accuracy: 0.9831
Epoch 3/5
4011/4011 [==============================] - 14s 4ms/sample - loss: 0.0343 - accuracy: 0.9913 - val_loss: 0.0448 - val_accuracy: 0.9860
Epoch 4/5
4011/4011 [==============================] - 15s 4ms/sample - loss: 0.0241 - accuracy: 0.9930 - val_loss: 0.0522 - val_accuracy: 0.9850


In [25]:
model.predict(matrix_test,batch_size=128)

array([[5.67734241e-05],
       [3.56912613e-04],
       [0.00000000e+00],
       [6.85453415e-07],
       [9.99984801e-01],
       [2.80737877e-05],
       [6.64532185e-04],
       [1.29759312e-04],
       [7.45058060e-07],
       [4.50313091e-05],
       [3.27825546e-06],
       [8.01682472e-06],
       [4.64916229e-06],
       [9.99996185e-01],
       [2.33799219e-04],
       [1.32841170e-02],
       [2.35438347e-06],
       [9.52482224e-05],
       [5.44548035e-04],
       [1.28805637e-04],
       [2.72780657e-04],
       [2.68220901e-06],
       [2.14141965e-01],
       [3.11055779e-03],
       [9.99999166e-01],
       [1.19209290e-06],
       [8.31484795e-06],
       [2.17318535e-04],
       [1.87217057e-01],
       [5.96046448e-08],
       [9.56851244e-03],
       [1.01327896e-06],
       [5.54323196e-05],
       [2.98023224e-08],
       [1.86979771e-03],
       [4.98768687e-03],
       [9.99999046e-01],
       [9.99999762e-01],
       [9.41872597e-04],
       [6.94394112e-06],


In [26]:
matrix_test.shape

(558, 150)